In [1]:
# Dependencies
import plotly.graph_objs as go
from shapely.geometry import shape
import plotly.express as px
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import json

### Migration flows by Region

Information about the data can be fount here: https://www.nomisweb.co.uk/sources/census_2021_mig

In [2]:
# Create df for migration into England and Wales
mig_rgn_in_df = pd.read_csv("./Resources/MIG004EW_RGN_IN.csv")
mig_rgn_in_df.head()

# Create df for emigration from England and Wales
mig_rgn_out_df = pd.read_csv("./Resources/MIG004EW_RGN_OUT.csv")
mig_rgn_out_df.head()

,Migrant region one year ago code,Migrant region one year ago label,Migration Region (outflow) (3 categories) code,Migration Region (outflow) (3 categories) label,Ethnic group (6 categories) code,Ethnic group (6 categories) label,National Statistics Socio-economic Classification (NS-SeC) (10 categories) code,National Statistics Socio-economic Classification (NS-SeC) (10 categories) label,Count
0,-8,Does not apply,-8,Does not apply,-8,Does not apply,-8,Does not apply,0
1,-8,Does not apply,-8,Does not apply,-8,Does not apply,1,"L1, L2 and L3: Higher managerial, administrati...",0
2,-8,Does not apply,-8,Does not apply,-8,Does not apply,2,"L4, L5 and L6: Lower managerial, administrativ...",0
3,-8,Does not apply,-8,Does not apply,-8,Does not apply,3,L7: Intermediate occupations,0
4,-8,Does not apply,-8,Does not apply,-8,Does not apply,4,L8 and L9: Small employers and own account wor...,0


In [3]:
# Count for regions in flow
mig_rgn_in = mig_rgn_in_df.groupby(['Migration Region (inflow) (6 categories) code', 'Migration Region (inflow) (6 categories) label'])['Count'].sum().reset_index()
mig_rgn_in = mig_rgn_in.sort_values(by='Count', ascending=False)

#Save df to csv file
mig_rgn_in.to_csv("./Resources/local_authority_in.csv", index=False)


# Count local regions out flow
mig_rgn_out = mig_rgn_out_df.groupby(['Migration Region (outflow) (3 categories) code', 'Migration Region (outflow) (3 categories) label'])['Count'].sum().reset_index()
mig_rgn_out = mig_rgn_out.sort_values(by='Count', ascending=False)


#Save df to csv file
mig_rgn_out.to_csv("./Resources/local_authority_out.csv", index=False)

# print(mig_rgn_out)

In [7]:
# Count for regions in flow
mig_rgn_in = mig_rgn_in_df.groupby(['Migration Region (inflow) (6 categories) code', 'Migration Region (inflow) (6 categories) label'])['Count'].sum().reset_index()
mig_rgn_in = mig_rgn_in.sort_values(by='Count', ascending=False)

# Rename "Does not apply" to "Didn't Move"
mig_rgn_out_df['Migration Region (outflow) (3 categories) label'] = mig_rgn_out_df['Migration Region (outflow) (3 categories) label'].replace('Does not apply', 'Didn\'t Move')

# Count local regions out flow
mig_rgn_out = mig_rgn_out_df.groupby(['Migration Region (outflow) (3 categories) code', 'Migration Region (outflow) (3 categories) label'])['Count'].sum().reset_index()
mig_rgn_out = mig_rgn_out.sort_values(by='Count', ascending=False)

# Save df to csv file
mig_rgn_in.to_csv("./Resources/local_authority_in.csv", index=False)
mig_rgn_out.to_csv("./Resources/local_authority_out.csv", index=False)


fig = go.Figure()

fig.add_trace(go.Bar(
    x=mig_rgn_in['Migration Region (inflow) (6 categories) label'],
    y=mig_rgn_in['Count'],
    name='Inflow'
))

fig.add_trace(go.Bar(
    x=mig_rgn_out['Migration Region (outflow) (3 categories) label'],
    y=mig_rgn_out['Count'],
    name='Outflow'
))

fig.update_layout(
    title='Migration Flow by Region',
    xaxis_tickfont_size=14,
    yaxis=dict(
        title='Count',
        titlefont_size=16,
        tickfont_size=14,
    ),
    legend=dict(
        x=0,
        y=1.0,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig.show()
